In [1]:
from utils import *
import pickle, csv, os
import numpy as np
from collections import defaultdict
from PatternHandler import PatternHandler
from DoublePropagation import DoublePropagation

Created file: /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/parameters.json
Parameters(
  base_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon
  data_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/data/parsed/five-three_5995.json
  lexicon_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/data/parsed/lexicon_6788.json
  output_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00
  output_targets_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/targets
  output_test_dir = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/test
  parameters_json_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/parameters.json
  output_time_txt_filepath = /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/elapsed_time.txt
  errlog_filepath = /home/d

In [2]:
base_dir = os.path.join(os.path.dirname(os.getcwd()), 'output', '20200716-10-07-00')
output_raw_df_pkl_filepath = os.path.join(base_dir, 'save', 'raw_df.pkl')
output_training_test_dfs_pkl_filepath = os.path.join(base_dir, 'save', '[%s]training_test_dfs.pkl')
output_dp_final_report_csv_filepath = os.path.join(base_dir, 'dp_final_report.csv')

raw_df = load_pkl(output_raw_df_pkl_filepath)
domains = raw_df['domain'].unique()
print(list(domains))

Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/raw_df.pkl
['MP3 player', 'DVD player', 'Digital camera2', 'Cell phone', 'Digital camera1', 'Wireless router', 'Speaker', 'Computer']


In [3]:
double_propagation, pattern_handler = DoublePropagation(), PatternHandler()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pr

In [4]:
kfold_results = defaultdict(lambda: [])
for domain in domains:
    filepath = output_training_test_dfs_pkl_filepath % domain
    if os.path.exists(filepath): 
        training_test_dfs = load_pkl(filepath)
        training_dfs, test_dfs = training_test_dfs[0], training_test_dfs[1]

        for df in test_dfs:
            df['predicted_targets'] = df.apply(lambda x: list(), axis=1)
            df['predicted_targets'] = df.apply(lambda x: pattern_handler.extract_targets_dp(double_propagation, x['doc'], x['opinion_words'], x['predicted_targets']), axis=1)

            pre_mul, rec_mul, pre_dis, rec_dis = calculate_precision_recall(df)
            f1_mul = calculate_f1(pre_mul,rec_mul)
            f1_dis = calculate_f1(pre_dis,rec_dis)
            kfold_results['_'.join([domain, 'Precision (multiple)'])].append(pre_mul)
            kfold_results['_'.join([domain, 'Recall (multiple)'])].append(rec_mul)
            kfold_results['_'.join([domain, 'F1 score (multiple)'])].append(f1_mul)
            
f, wr = start_csv(output_dp_final_report_csv_filepath)
wr.writerow(['Domain', 'Measure', 'DP (R11+R12)'])
for domain in domains:
    for measure in ['Precision (multiple)', 'Recall (multiple)', 'F1 score (multiple)']:
        wr.writerow([domain, measure, '%.4f'%np.mean(kfold_results['_'.join([domain, measure])])])
end_csv(f, output_dp_final_report_csv_filepath)

Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[MP3 player]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[DVD player]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[Digital camera2]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[Cell phone]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[Digital camera1]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[Wireless router]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[Speaker]training_test_dfs.pkl
Loaded /home/dmlab/jihye/GIT/domain-specific-sentiment-lexicon/output/20200716-10-07-00/save/[Compu